In [6]:
import urllib.request as ur # import para obtener la página web 
import re                   # import para utilizar expresiones regulares 

# Obtener página web con incidencias de Madrid 
f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=13&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')
s = f.read().decode()
f.close()


In [7]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [8]:
from pprint import pprint     # import para hacer prints boninitos 

dividedByTr = s.split("<tr") # Dividir el string por tr para incidencias con hora fin 
endTimeElements = [] # Almacenará elementos que tienen hora fin 

# Obtener las incidencias que tienen hora fin 
print("Incidencias con hora fin")
endTime = re.compile(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",re.DOTALL)
for x in dividedByTr:
    confin = endTime.findall(x)
    #confin = re.findall(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",x,re.DOTALL)
    if len(confin): # Si el elemento contiene algún dato se agrega a la lista de elementos con hora fin
        endTimeElements.append(confin[0]) 
        pprint(confin)

# Obtener todas las incidencias 
print("\nTodas las incidencias") # Las cuales incluyen las que tienen hora fin pero no incluyen ese dato
conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d)?.*?(\d\d/\d\d/\d\d\d\d)?.*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(conjunto)



Incidencias con hora fin
[('18:00',
  '06/03/2019',
  '21:00',
  '06/03/2019',
  'M-50',
  'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por <b>CIRCULACIÓN</b> en<b> '
  'Túneles Boadilla - Valdepastores</b>: </span> <span style="margin-top:10px; '
  'float:left; clear:both">  -  La carretera  <b> <span '
  'style="color:#ab3000">M-50 </span> </b> a la altura de <b>BOADILLA DEL '
  'MONTE (MADRID)</b> desde el <b> km 77.7</b> al <b> km 72.7</b> sentido '
  '<b>DECRECIENTE DE LA KILOMETRACIÓN</b> hacia <b>Decreciente')]
[('18:00',
  '06/03/2019',
  '21:00',
  '06/03/2019',
  'M-506',
  'RESTRICCIONES A VEHÍCULOS QUE NECESITAN AUTORIZACIÓN COMPLEMENTARIA</b> por '
  '<b>CIRCULACIÓN</b> en<b> Acceso a Madrid M-506</b>: </span> <span '
  'style="margin-top:10px; float:left; clear:both">  -  La carretera  <b> '
  '<span style="color:#ab3000">M-506 </span> </b> desde el <b> km 0 </b> en '
  '<b>VILLAVICIOSA DE ODON (MADRID)</b> al <b> km 53</b> en <b>ARGANDA '
  '(MADRID)</b> sentido <b>AMBOS S

[('06:00',
  '06/03/2019',
  '23:00',
  '06/03/2019',
  'M-111',
  'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por <b>CIRCULACIÓN</b> en<b> '
  'Túnel Barajas</b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  -  La carretera  <b> <span style="color:#ab3000">M-111 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 1.1</b> '
  'al <b> km 5</b> sentido <b>AMBOS SENTIDOS')]

Todas las incidencias
[('18:48',
  '06/03/2019',
  '',
  '',
  'M-40',
  'RETENCIÓN / CONGESTIÓN</b> por <b>ACCIDENTE</b> y <b>CIRCULACIÓN</b> con '
  'circulación irregular  en<b></b>: </span> <span style="margin-top:10px; '
  'float:left; clear:both">  - La AUTOPISTA / AUTOVÍA <b> <span '
  'style="color:#ab3000">M-40 </span> </b> a la altura de <b>MADRID '
  '(MADRID)</b> desde el <b> km 5</b> al <b> km 8</b> sentido <b>DECRECIENTE '
  'DE LA KILOMETRACIÓN</b> hacia <b>A-6'),
 ('18:48',
  '06/03/2019',
  '',
  '',
  'A-42',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCULACIÓN<

In [9]:
diccionario = {} # variable que se utilizará para almacenar las incidencias una sola vez 

for x in range(len(endTimeElements)): # se ejecuta para cada elemento con hora fin 
    identificador = endTimeElements[x][0] + endTimeElements[x][1] + endTimeElements[x][4]  # concatenación de valores para crear identificador para diccionario
    diccionario.update({identificador : endTimeElements[x]}) # agregación de valores al diccionario

for x in range(len(conjunto)): # se ejecuta para cada elemento
    identificador = conjunto[x][0] + conjunto[x][1] + conjunto[x][4] # concatenación de valores para crear identificador para diccionario
    if diccionario.get(identificador) == None:    # En caso de que el identificador exista, salta este paso (es decir, el elemento tenía hora fin)
        diccionario.update({identificador : conjunto[x]}) # Si el identificador no existe, se crea un nuevo elemento en el diccionario
'''
Para todas las carreteras
Indica que hubo un atasco si la incidencia tiene hora fin 
Indica que existe un atasco si la incidencia no tiene hora fin 
'''
        
for x in diccionario:
    valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
    fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
    cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
    if valores[2] != '':  # En caso de que exista una hora fin
        mensaje = "Hubo un atasco en la carretera " + valores[4]
        fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
    else: 
        mensaje = "Existe un atasco en la carretera " + valores[4]
    mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
    mensaje = mensaje + fin
    mensaje = mensaje + " con la siguientes características:\n" +  cleantext
    
    print(mensaje+"\n")
    


Hubo un atasco en la carretera M-50 que inicio a las 18:00 el día 06/03/2019 el cual terminó a las 21:00 el día 06/03/2019 con la siguientes características:
RESTRICCIONES A VEHÍCULOS QUE NECESITAN AUTORIZACIÓN COMPLEMENTARIA por CIRCULACIÓN en Acceso a Madrid M-50:    -  La carretera   M-50   desde el  km 0  en SAN SEBASTIAN DE LOS REYES (MADRID) al  km 84.5 en ROZAS DE MADRID (LAS) (MADRID) sentido AMBOS SENTIDOS

Hubo un atasco en la carretera M-506 que inicio a las 18:00 el día 06/03/2019 el cual terminó a las 21:00 el día 06/03/2019 con la siguientes características:
RESTRICCIONES A VEHÍCULOS QUE NECESITAN AUTORIZACIÓN COMPLEMENTARIA por CIRCULACIÓN en Acceso a Madrid M-506:    -  La carretera   M-506   desde el  km 0  en VILLAVICIOSA DE ODON (MADRID) al  km 53 en ARGANDA (MADRID) sentido AMBOS SENTIDOS

Hubo un atasco en la carretera M-45 que inicio a las 18:00 el día 06/03/2019 el cual terminó a las 21:00 el día 06/03/2019 con la siguientes características:
RESTRICCIONES A VEHÍC

In [5]:
'''
Para la carretera M-40 
Indica que hubo un atasco si la incidencia tiene hora fin 
Indica que existe un atasco si la incidencia no tiene hora fin 
'''

for x in diccionario:
    if "M-40" in x: 
        valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
        fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
        cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
        if valores[2] != '':  # En caso de que exista una hora fin
            mensaje = "Hubo un atasco en la carretera " + valores[4]
            fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
        else: 
            mensaje = "Existe un atasco en la carretera " + valores[4]
        mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
        mensaje = mensaje + fin
        mensaje = mensaje + " con la siguientes características:\n" +  cleantext

        print(mensaje+"\n")
    


Existe un atasco en la carretera M-40 que inicio a las 10:23 el día 06/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 32 al  km 28 sentido CRECIENTE DE LA KILOMETRACIÓN hacia A-6

Existe un atasco en la carretera M-40 que inicio a las 09:28 el día 06/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 21 al  km 26 sentido DECRECIENTE DE LA KILOMETRACIÓN hacia A-4

